## 🤖 Ders 2 Uygulaması: Kendi Evrenini Yarat - Grid World Simülasyonu

**Pekiştirmeli Öğrenme (Reinforcement Learning)** teorisini sadece okumakla kalmayıp, kendi ellerimizle koda döktüğümüz interaktif bir Python simülasyonu hazırladık ! 

Ders 2 notlarında incelediğimiz **"Grid World (Izgara Dünyası)"** senaryosunu temel alan bu projede, hiçbir şey bilmeyen bir yapay zeka ajanını (agent) labirente bırakıyor ve hatalarından ders çıkararak en iyi yolu bulmasını izliyoruz.

###  Bu Uygulamada Neler Var?

Bu kod, sabit bir labirent yerine kurallarını tamamen **sizin belirlediğiniz** dinamik bir evren sunar:
* **Haritayı Siz Çizin:** Izgara boyutunu (Örn: 5x5, 10x10) ve ajanın başlangıç noktasını kendiniz belirleyebilirsiniz.
* **Tuzakları ve Ödülleri Yerleştirin:** Büyük ödülün ve oyunu bitiren ölümcül tuzağın yerini seçerek ajanı zorlayabilirsiniz.
* **Hareket Maliyetini Ayarlayın:** Ajanın her adımda yiyeceği cezayı değiştirerek, onun hedefe giden en kısa yolu bulmasını (veya sonsuza kadar dolanmasını) sağlayabilirsiniz.

###  Ajan Nasıl Öğreniyor?

1. **Eğitim Kampı (Keşif ve Sömürü):** Ajan başlangıçta tamamen rastgele hareket ederek çevreyi keşfeder (Explore) . Zamanla Q-Tablosunu (hafızasını) doldurdukça, öğrendiği kârlı yolları kullanmaya (Exploit) başlar .
2. **Gecikmeli Ödül (Aggregate Reward):** Ajan anlık cezalara takılmadan, oyun sonundaki toplam getirisini maksimize etmeyi öğrenir .
3. **Görselleştirme:** Eğitim bittiğinde, `matplotlib` ve `seaborn` kütüphaneleri kullanılarak ajanın çıkardığı kusursuz kılavuz, yani **"Optimal Politika"** ($\pi^*(s)$) renkli bir harita üzerinde yön oklarıyla çizdirilir.

###  Nasıl Çalıştırılır?

1. Bilgisayarınızda `numpy`, `matplotlib` ve `seaborn` kütüphanelerinin kurulu olduğundan emin olun (`pip install numpy matplotlib seaborn`).
2. İlgili klasördeki `Jupyter Notebook` dosyasını çalıştırın.
3. Konsolda size sorulan sorulara cevap vererek evreninizi yaratın (boş bırakıp Enter'a basarak varsayılan kuralları da kullanabilirsiniz).
4. Eğitimin bitmesini bekleyin ve ajanın bulduğu kırmızı rotanın tadını çıkarın!

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

def get_user_input(prompt, default_value):
    """Kullanıcı 'Enter'a basıp geçerse varsayılan değeri atayan minik asistanımız."""
    user_input = input(f"{prompt} (Varsayılan: {default_value}): ")
    return int(user_input) if user_input.strip() else default_value

print("--- GÖRSEL PEKİŞTİRMELİ ÖĞRENME: KENDİ EVRENİNİ OLUŞTUR ---")
print("İpucu: Soruları boş bırakıp 'Enter'a basarsanız standart kurallar geçerli olur.\n")

# --- 1. DÜNYAYI VE KURALLARI İNŞA EDİYORUZ ---
GRID_SIZE = get_user_input("1. Izgara Boyutunu belirleyin (Örn: 4, 5, 6)", 5)
NUM_STATES = GRID_SIZE * GRID_SIZE
NUM_ACTIONS = 4

START_STATE = get_user_input("2. Ajanın Başlangıç Karesi", 0)
GOAL_STATE = get_user_input("3. Büyük Ödülün (Hedef) Karesi", NUM_STATES - 1)
GOAL_REWARD = get_user_input("4. Hedefe Ulaşma Ödülü (Puan)", 40)
SEVERE_TRAP = get_user_input("5. Büyük Tuzağın Karesi (Oyun Bitirici)", (NUM_STATES // 2) - 1)
TRAP_PENALTY = get_user_input("6. Tuzağa Düşme Cezası (Eksi Puan olarak)", -20)
LIVING_PENALTY = get_user_input("7. Adım Başı Ceza (Hareket Maliyeti)", -1)

# --- 2. AJANIN BEYNİ (Q-TABLOSU) ---
q_table = np.zeros((NUM_STATES, NUM_ACTIONS))

# --- 3. ÖĞRENME PSİKOLOJİSİ (HİPERPARAMETRELER) ---
learning_rate = 0.1      
discount_factor = 0.9    
epsilon = 1.0            
epsilon_decay = 0.995    
min_epsilon = 0.01       
episodes = 2000          

def step(state, action):
    """Ajanın fiziksel olarak adım attığı ve karşılığında çevreden tokat veya şeker yediği yer."""
    row = state // GRID_SIZE
    col = state % GRID_SIZE
    
    if action == 0 and col > 0: col -= 1          
    elif action == 1 and col < GRID_SIZE - 1: col += 1  
    elif action == 2 and row > 0: row -= 1          
    elif action == 3 and row < GRID_SIZE - 1: row += 1  
    
    new_state = row * GRID_SIZE + col
    
    if new_state == GOAL_STATE:
        return new_state, GOAL_REWARD, True   
    elif new_state == SEVERE_TRAP:
        return new_state, TRAP_PENALTY, True  
    else:
        return new_state, LIVING_PENALTY, False 

# --- 4. EĞİTİM KAMPI BAŞLIYOR ---
print("\nAjan labirente bırakıldı... Kendi kendine öğreniyor, lütfen bekleyin...")
for episode in range(episodes):
    state = START_STATE
    done = False
    step_count = 0 
    
    while not done and step_count < 100:
        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, NUM_ACTIONS - 1)
        else:
            action = np.argmax(q_table[state, :])
            
        new_state, reward, done = step(state, action)
        
        max_future_q = np.max(q_table[new_state, :]) 
        current_q = q_table[state, action]           
        
        new_q = current_q + learning_rate * (reward + discount_factor * max_future_q - current_q)
        q_table[state, action] = new_q
        
        state = new_state
        step_count += 1
        
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

print("Eğitim Tamamlandı! Ajan labirentin efendisi oldu.")

# --- 5. SINAV ANI: ÖĞRENDİĞİ EN İYİ ROTAYI VE PUANINI BULALIM ---
optimal_path = set()
curr_state = START_STATE
step_limit = 0
total_score = 0  # Ajanın topladığı puanları burada biriktireceğiz

print("\n--- SINAV SONUÇLARI ---")

while curr_state != GOAL_STATE and curr_state != SEVERE_TRAP and step_limit < NUM_STATES * 2:
    optimal_path.add(curr_state)
    best_act = np.argmax(q_table[curr_state, :])
    
    # Adımı at, yeni kareyi ve o karedeki ÖDÜLÜ/CEZAYI (reward) al
    curr_state, reward, _ = step(curr_state, best_act)
    
    # Cebe giren puanı toplam puana ekle
    total_score += reward
    step_limit += 1

print(f"Toplam Atılan Adım: {step_limit}")
print(f"Ajanın Bu Kusursuz Rotadan Topladığı Net Puan: {total_score}")
if total_score > 0:
    print("Yorum: Harika! Ajan yolda yediği -1 cezalara rağmen günün sonunda kâra geçti.")
else:
    print("Yorum: Ajan hedefe ulaşsa bile yolda çok fazla adım attığı veya tuzağa düştüğü için zararda!")

# --- 6. GÖRSEL ŞÖLEN (MATPLOTLIB) ---
print("\nHarita çiziliyor...")
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_xlim(0, GRID_SIZE)
ax.set_ylim(0, GRID_SIZE)
ax.invert_yaxis()

grid_matrix = np.zeros((GRID_SIZE, GRID_SIZE))
sns.heatmap(grid_matrix, cmap="Blues", cbar=False, linewidths=2, linecolor='black', ax=ax, alpha=0.1)

action_arrows = {
    0: (-0.3, 0),    
    1: (0.3, 0),     
    2: (0, -0.3),    
    3: (0, 0.3)      
}

for state in range(NUM_STATES):
    row, col = state // GRID_SIZE, state % GRID_SIZE
    center_x, center_y = col + 0.5, row + 0.5
    
    if state == START_STATE:
        ax.text(center_x, center_y - 0.3, "BAŞLANGIÇ", ha='center', va='center', color='blue', fontweight='bold', fontsize=9)
    
    if state == GOAL_STATE:
        ax.add_patch(plt.Rectangle((col, row), 1, 1, color='lightgreen'))
        ax.text(center_x, center_y, f"ÖDÜL\n(+{GOAL_REWARD})", ha='center', va='center', color='darkgreen', fontweight='bold')
        continue 
        
    if state == SEVERE_TRAP:
        ax.add_patch(plt.Rectangle((col, row), 1, 1, color='salmon'))
        ax.text(center_x, center_y, f"TUZAK\n({TRAP_PENALTY})", ha='center', va='center', color='darkred', fontweight='bold')
        continue 

    best_action = np.argmax(q_table[state, :])
    
    if np.max(q_table[state, :]) == 0 and np.min(q_table[state, :]) == 0:
        continue 
        
    dx, dy = action_arrows[best_action]
    
    is_main_path = state in optimal_path
    
    arrow_color = 'red' if is_main_path else 'lightgray'
    arrow_width = 0.15 if is_main_path else 0.05
    line_width = 3 if is_main_path else 1
    
    ax.arrow(center_x - dx/2, center_y - dy/2, dx, dy, 
             head_width=arrow_width, head_length=arrow_width, 
             fc=arrow_color, ec=arrow_color, lw=line_width)

plt.title(f"Ajanın Bulduğu Ana Rota (Toplam Puan: {total_score})", fontsize=14, fontweight='bold', pad=15)
plt.axis('off') 
plt.show()